### **<span style="color:crimson">INNER `JOIN`<span>**
* If you have a column that links table A to table B you can use inner join.
* Finds intersection of two datasets based on a link and returns you that subset.

<img src="../../images/inner-join.png" height="150">

___

### **<span style="color:crimson">Example Dataset</span>**

**DATABASE: `Employees`**

\
**TABLE A: `employees`**

| emp_no | first_name | last_name |
|--------|------------|-----------|
| 1      | John       | Doe       |
| 2      | Jane       | Smith     |

\
**TABLE B: `salaries`**

| emp_no | salary   | from_date  |
|--------|----------|------------|
| 1      | 50000    | 2023-01-01 |
| 2      | 60000    | 2023-01-01 |

\
**TABLE C: `titles`**

| emp_no | title         | from_date  |
|--------|---------------|------------|
| 1      | Junior Engineer | 2023-01-01 |
| 2      | Senior Engineer | 2023-01-01 |
___

#### **<span style="color:darkorange">INNER `JOIN`: LINKING 2 TABLES</span>**

```sql
SELECT a.emp_no,
       CONCAT(a.first_name, a.last_name) AS "name",
       b.salary
FROM employees AS a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
ORDER BY a.emp_no ASC;
```

\
<span style="color:mediumseagreen">**Result:**</span>

| emp_no | name      | salary |
|--------|-----------|--------|
| 1      | JohnDoe   | 50000  |
| 2      | JaneSmith | 60000  |

\
<span style="color:mediumseagreen">**Notes:**</span>

- Combines only the rows from `employees` and `salaries` where the `emp_no` values match.
- Same as Multi Table `SELECT` (with a link).
- Best practice. More readable than multi table `SELECT`.
___

#### **<span style="color:darkorange">INNER `JOIN`: LINKING MULTIPLE TABLES</span>**

### <span style="color:hotpink">What if we wanted to know the raises from a promotion?</span>
- <span style="color:hotpink">Not a percentage based raise, but one that leads to a promotion in title.</span>
- <span style="color:hotpink">Title changes always follow two days after a promotion.</span>

#### **<span style="color:mediumseagreen">Let's try joining everything based on emp_no...</span>**

```sql
SELECT a.emp_no, b.salary, b.from_date, c.title
FROM employees as a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
INNER JOIN titles AS c ON c.emp_no = a.emp_no
ORDER BY a.emp_no ASC, b.from_date ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

| emp_no | salary | from_date  | title           |
|--------|--------|------------|-----------------|
| 1      | 50000  | 2023-01-01 | Junior Engineer |
| 1      | 50000  | 2023-01-01 | Senior Engineer |
| 1      | 50000  | 2023-01-03 | Senior Engineer |
| 1      | 50000  | 2023-01-03 | Manager          |
| 2      | 60000  | 2023-01-01 | Senior Engineer |
| 2      | 60000  | 2023-01-01 | Manager          |

\
**<span style="color:mediumseagreen">Notes:</span>**
- The result set does not effectively represent the relationship between raises and title changes.
- The query retrieves all combinations of salaries and titles for each employee, leading to redundant data.
- The titles are not filtered based on uniqueness, resulting in confusing and unnecessary rows for a single employee.

#### **<span style="color:mediumseagreen">Let's try a refined approach...</span>**

```sql
SELECT a.emp_no, b.salary, b.from_date, c.title
FROM employees as a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
INNER JOIN titles AS c ON c.emp_no = a.emp_no 
AND b.from_date = c.from_date
ORDER BY a.emp_no ASC, b.from_date ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

| emp_no | salary | from_date  | title           |
|--------|--------|------------|-----------------|
| 1      | 50000  | 2023-01-01 | Junior Engineer |
| 2      | 60000  | 2023-01-01 | Senior Engineer |

\
**<span style="color:mediumseagreen">Notes:</span>**
- The refined query addresses the redundancy issue by joining the `titles` table based on both `emp_no` and `from_date`, ensuring that only the relevant title for a specific employee and date is included.
- This approach provides a clearer representation of promotions without unnecessary data duplication.

#### **<span style="color:mediumseagreen">Now we need to correct our filter to account for the two day delay in title change...</span>**

```sql
SELECT a.emp_no, b.salary, b.from_date, c.title
FROM employees as a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
INNER JOIN titles AS c ON c.emp_no = a.emp_no 
AND (b.from_date + INTERVAL '2 days') = c.from_date
ORDER BY a.emp_no ASC, b.from_date ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

| emp_no | salary | from_date  | title           |
|--------|--------|------------|-----------------|
| 1      | 50000  | 2023-01-03 | Manager          |
| 2      | 60000  | 2023-01-03 | Manager          |

\
**<span style="color:mediumseagreen">Notes:</span>**
- The corrected query filters titles based on a two-day interval after a salary change.
- Now we get all of the employees that ever got a title change.
- Getting the intersection of all of the data for the titles that matched a raise.

### <span style="color:hotpink">What if we want to know the original salary and the salary at promotion?</span>

#### **<span style="color:mediumseagreen">Let's get the original and promoted salary...</span>**

```sql
SELECT a.emp_no, b.salary, b.from_date, c.title
FROM employees as a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
INNER JOIN titles AS c ON c.emp_no = a.emp_no 
    AND (
        b.from_date = c.from_date
        OR (b.from_date + INTERVAL '2 days') = c.from_date
    )
ORDER BY a.emp_no ASC, b.from_date ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

| emp_no | salary | from_date  | title           |
|--------|--------|------------|-----------------|
| 1      | 50000  | 2023-01-01 | Junior Engineer |
| 1      | 50000  | 2023-01-03 | Senior Engineer |
| 2      | 60000  | 2023-01-01 | Senior Engineer |
| 2      | 60000  | 2023-01-03 | Manager          |

\
**<span style="color:mediumseagreen">Explanation:</span>**
- Retrieves information about the original salary when an employee joined and the salary at each promotion.
- Includes both starting salaries and salaries at each title change.

#### **<span style="color:mediumseagreen">We can improve our query by adding employee names and handling missing titles...</span>**

```sql
SELECT a.emp_no,
       CONCAT(a.first_name, a.last_name) AS "name",
       b.salary,
       COALESCE(c.title, 'No title change'),
       COALESCE(c.from_date::text, '-') AS "Title taken on"
FROM employees AS a
INNER JOIN salaries AS b ON a.emp_no = b.emp_no
INNER JOIN titles AS c 
ON  c.emp_no = a.emp_no AND (
    c.from_date = b.from_date OR 
    c.from_date = (b.from_date + INTERVAL '2 days')
)
ORDER BY a.emp_no ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

| emp_no | name      | salary | title           | Title taken on |
|--------|-----------|--------|-----------------|-----------------|
| 1      | JohnDoe   | 50000  | Junior Engineer | 2023-01-01      |
| 1      | JohnDoe   | 50000  | Senior Engineer | 2023-01-03      |
| 2      | JaneSmith | 60000  | Senior Engineer | 2023-01-01      |
| 2      | JaneSmith | 60000  | Manager          | 2023-01-03      |

\
**<span style="color:mediumseagreen">Explanation:</span>**
- Improved query that includes employee names and handles cases where there might be missing titles.
- Uses `COALESCE` to display "No title change" when there is no title change information available.
- Presents a more organized and summarized view of the data, including employee names.
___